In [1]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model_path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-6/checkpoint-4"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_4bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = torch.nn.DataParallel(model)

model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [3]:
import json

json_file_path = '../../ExTES.json'  # Change 'output.json' to your desired file name

# Write the list to the JSON file
with open(json_file_path, 'rb') as json_file:
    extes_data = json.load(json_file)
    
file_path = '.././scenes.txt'  # Replace 'your_text_file.txt' with the actual path to your text file

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read all lines from the file and split them based on newlines
    scenes = file.read().splitlines()

# Now, 'lines' is a list containing each line from the file
print(scenes)


['communication challenges', 'breakups or divorce', 'difficulties in a friendship', 'coping with the illness or death of a loved one', 'work-related stress and burnout', 'financial worries and uncertainty', 'academic stress', 'depression and low mood', 'managing bipolar disorder', 'anxiety and panic', 'parenthood and parenting challenges', 'low self-esteem or lack of confidence', 'body image concerns and eating disorders', 'coping with a diagnosis or medical treatment', 'addiction and recovery', 'coping with the illness of a family member', 'feeling overwhelmed and stressed', 'grief and loss', 'healing from sexual assault or violence', 'surviving and recovering from physical or emotional abuse', 'healing from abuse or domestic violence']


In [4]:
emotion_data = []
not_available = []
for i in range(len(extes_data)):
    try:
        if (extes_data[i]['scene'].strip().lower() or extes_data[i]['description'].strip().lower()) in scenes:
            
            emotion_data.append(extes_data[i])
            
    except:
        not_available.append(i)
        
def create_list(data, value):
    s = []
    for i in range(len(data)):
        #print(value.lower())
        if data[i]['scene'].lower().rstrip() == value.lower():
            s.append(data[i])
            
    return s
    
combined_scenes = []
for i in range(0,21):
    
    s = create_list(emotion_data, scenes[i])
    if len(s)>200:
        combined_scenes.append(s)

combined_scenes[0] = combined_scenes[0][0:1000]

In [5]:
#item  = combined_scenes[0][0]

formatted_data1 = []

for i in range(len(combined_scenes)):
    for item in combined_scenes[i]:
        formatted_data = []
        scene = item['scene'].lower()
        #description = item['description']
        content = item['content']
        """formatted_data.append({"scene": scene, "role": "scene"})
        if item['description']:
            formatted_data.append({"description": description, "role": "description"})"""
        
        formatted_data.append({"scene": scene, "role": "scene"})
        for element in content:
            for role, content_text in element.items():
                if role=="User":
                    role = "user"
                elif role=="AI":
                    role = "assistant"
                else:
                    continue    
                    
                formatted_data.append({"role": role, "content": content_text})
                
        formatted_data1.append(formatted_data)

In [6]:
import pandas as pd

# Define column names
columns = ['scene', 'content']

# Create an empty DataFrame with specified columns
df = pd.DataFrame(columns=columns)

# Display the empty DataFrame
print(df)

for i in range(len(formatted_data1)):
    empty_dict = {}
    empty_dict['scene'] = formatted_data1[i][0]['scene']
    #empty_dict['description'] = formatted_data1[i][1]['description']  
    empty_dict['content'] = formatted_data1[i]
    
    df = df.append(empty_dict, ignore_index=True)

Empty DataFrame
Columns: [scene, content]
Index: []


/tmp/ipykernel_2276963/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_2276963/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_2276963/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_2276963/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_2276963/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be remove

In [7]:
df.dropna(inplace=True)

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(df, df['scene']):
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    
X_train['content'] = X_train['content'].apply(lambda x: x[1:])
X_test['content'] = X_test['content'].apply(lambda x: x[1:])

from datasets import DatasetDict
from datasets import Dataset
datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(X_train),
    "test": Dataset.from_pandas(X_test)
    })

/tmp/ipykernel_2276963/875359953.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['content'] = X_train['content'].apply(lambda x: x[1:])
/tmp/ipykernel_2276963/875359953.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['content'] = X_test['content'].apply(lambda x: x[1:])


In [9]:
import ast 
def apply_chat_template(example, tokenizer):
    messages = (example["content"])
    try:
        #print("tokenized")
        tokenized_content = tokenizer.apply_chat_template(messages, tokenize=False)
        return {'content': tokenized_content}
    except Exception as e:
       #print(f"Unable to tokenize:")
        text =  "None"
        return {'content': " "}

In [10]:
import re
import random
from multiprocessing import cpu_count
column_names = ['scene','content']
raw_datasets = datasets_train_test.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

Applying chat template (num_proc=112):   0%|          | 0/2585 [00:00<?, ? examples/s]

Applying chat template (num_proc=112):   0%|          | 0/647 [00:00<?, ? examples/s]

In [11]:
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

In [12]:
torch.cuda.device_count()

3

In [26]:
device_map = {"cuda": torch.cuda.current_device()} if torch.cuda.is_available() else None
print(device_map)

{'cuda': 0}


In [ ]:
device_map = {'cuda:0': 1, 'cuda:1': 2, 'cuda:2': 3}  # Map CUDA devices to indices

# Set the device map
for device, index in device_map.items():
    torch.cuda.set_device(device)


In [13]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="bfloat16",
)


model_kwargs = dict(
    attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map="auto",
    quantization_config=quantization_config,
)

In [14]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments

# path where the Trainer will save its checkpoints and logs
output_dir = '/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-4'

# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=4,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=8, # originally set to 8
    # push_to_hub=True,
    # hub_model_id="zephyr-7b-sft-lora",
    # hub_strategy="every_save",
    # report_to="tensorboard",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
    
)

# based on config
peft_config = LoraConfig(
        r=32,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)



In [15]:
def preprocess_dataset(dataset):
    for sample in dataset:
        # Convert 'content' field to string
        sample['content'] = str(sample['content'])
    return dataset

# Preprocess datasets
train_dataset = preprocess_dataset(train_dataset)
eval_dataset = preprocess_dataset(eval_dataset)



In [16]:
# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Then, create the trainer
trainer = SFTTrainer(
        model=model,
        #model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="content",
        tokenizer=tokenizer,
        packing=True,  # Disable packing
        peft_config=peft_config,
        max_seq_length=4096,
    )


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Using auto half precision backend


In [38]:
"""device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Assuming `trainer` is your trainer object and `model` is an attribute of the trainer
trainer.model = trainer.model.to(device)"""

cuda


In [42]:
parameters = trainer.model.state_dict()

# Visualize the parameters
print(parameters.keys())

odict_keys(['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_map', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.absmax', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.quant_map', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.w

In [43]:
trainer.train()

***** Running training *****
  Num examples = 447
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 128


  Gradient Accumulation steps = 128
  Total optimization steps = 12
  Number of trainable parameters = 27,262,976


Epoch,Training Loss,Validation Loss
0,No log,2.185255
1,2.286100,1.937833
2,1.901000,1.828836
3,1.901000,1.822844


***** Running Evaluation *****
  Num examples = 113
  Batch size = 1
***** Running Evaluation *****
  Num examples = 113
  Batch size = 1
***** Running Evaluation *****
  Num examples = 113
  Batch size = 1
***** Running Evaluation *****
  Num examples = 113
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12, training_loss=2.0478906631469727, metrics={'train_runtime': 3390.9152, 'train_samples_per_second': 0.527, 'train_steps_per_second': 0.004, 'total_flos': 2.69447573633237e+17, 'train_loss': 2.0478906631469727, 'epoch': 3.44})

In [17]:
trainer.train()

***** Running training *****
  Num examples = 447
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 1,024
  Gradient Accumulation steps = 128
  Total optimization steps = 4
  Number of trainable parameters = 27,262,976
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 GiB (GPU 1; 79.15 GiB total capacity; 42.00 GiB already allocated; 5.14 GiB free; 69.55 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [44]:
trainer.save_model('/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-4')

Saving model checkpoint to /home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-4
/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error 403 Client Error. (Request ID: Root=1-66284db0-372b518102bc58e41ce5f77d;d60320f3-6ab7-44be-ab2e-709701fa935c)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2 to ask for access. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.2.
  warnings.warn(
/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.2 - will assume that

In [55]:
messages = [ {
  'role':'user',
  'content': 'Hey, are you free? I need someone to talk to.',
}  ]

In [56]:
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = trainer.model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

[INST] Hey, are you free? I need someone to talk to. [/INST] Of course! I'm here to listen. Feel free to share what's on your mind and I'll do my best to provide a helpful and supportive response. Remember, I'm an AI language model, so I don't have feelings or emotions, but I can help you process yours. Let's chat! What's been going on?


: 

In [51]:
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])

 I'm sorry to hear that you've been feeling down and struggling with your mental health. I can understand how overwhelming and isolating it can feel, and it's important to remember that you're not alone in your experience. It takes a lot of courage to reach out for support, and I'm here to listen and offer what help I can.

If you feel comfortable, you can share more about what's been causing you to feel this way. Sometimes just talking about our feelings and thoughts can help us gain perspective and feel less alone. And if you don't feel ready to talk about specifics, that's okay too. Just knowing that someone is there to listen and care can be a source of comfort.

There are also many resources available that can provide additional support and help. I can suggest some options if you'd like. And remember, it's okay to ask for help when you need it. Taking care of your mental health is an important part of taking care of yourself, and there's no shame in seeking support when things get